In [1]:

# --- Define ITEMIDs (EXTENDED) ---
feature_itemids = {
    "Temperature (F)": [223761],
    "Heart Rate": [220045],
    "Respiratory Rate": [220210],
    "Systolic Blood Pressure": [220179],
    "Diastolic Blood Pressure": [220180],
    "Pulse Oximetry": [220277],
    "Troponin": [227429],
    "BUN": [225624],
    "INR": [227467],
    "PTT": [220339],
    "Creatinine": [220615],
    "Glucose": [220621],
    "Sodium": [[220645, 227442][0]],
    "Potassium": [[220689, 227451][0]],
    "Chloride": [220602],
    "PEEP Set": [224700],
    "Tidal Volume": [224685],
    "Anion Gap": [227073],
    "Inspired O2 Fraction": [223835],
    
    "RBC": [[220546, 813, 1120, 1532][0]],
    "WBC": [[220545, 811, 1530][0]],
    "Platelets": [[227451, 814, 1534][0]],
    "Hemoglobin": [[22028, 815, 1118, 1531][0]],
    "Hematocrit": [[220549, 807, 1379, 1119][0]],
    
    "Bands": [[225943, 827][0]],
    "Neutrophils": [[220573, 829, 1138][0]],
}

In [2]:
import pandas as pd

# --- Load Data ---
icustays      = pd.read_csv("../MIMIC_resources/ICUSTAYS.csv", usecols=["SUBJECT_ID", "HADM_ID", "ICUSTAY_ID"])
chartevents   = pd.read_csv("../MIMIC_resources/CHARTEVENTS_first_only.csv", usecols=["SUBJECT_ID", "HADM_ID", "ICUSTAY_ID", "ITEMID", "VALUENUM"])
admissions    = pd.read_csv("../MIMIC_resources/ADMISSIONS.csv", usecols=["SUBJECT_ID", "HADM_ID", "ADMISSION_TYPE", "ADMITTIME"])
prescriptions = pd.read_csv("../MIMIC_resources/PRESCRIPTIONS.csv", usecols=["subject_id", "hadm_id", "drug"])  # Lowercase column names
patients      = pd.read_csv("../MIMIC_resources/PATIENTS.csv", usecols=["SUBJECT_ID", "GENDER", "DOB"])
d_items       = pd.read_csv("../MIMIC_resources/D_ITEMS.csv")

# --- Filter CHARTEVENTS ---
chartevents = chartevents.merge(icustays, on=["SUBJECT_ID", "HADM_ID", "ICUSTAY_ID"], how="inner")

# --- Filter CHARTEVENTS ---
# filtered_chartevents = chartevents[chartevents["ITEMID"].isin([item for sublist in feature_itemids.values() for item in sublist])]
filtered_chartevents = chartevents[chartevents["ITEMID"].isin([item for sublist in feature_itemids.values() for item in sublist])]

# --- Pivot Vitals and Labs ---
pivot_vitals = filtered_chartevents.pivot_table(index=["SUBJECT_ID", "HADM_ID", "ICUSTAY_ID"], columns="ITEMID", values="VALUENUM", aggfunc="mean")
itemid_to_feature = {v[0]: k for k, v in feature_itemids.items()}
pivot_vitals.rename(columns=itemid_to_feature, inplace=True)

# --- Process Admissions ---
admissions["ADM_ELECTIVE"] = (admissions["ADMISSION_TYPE"] == "ELECTIVE").astype(int)
admissions["ADM_EMERGENCY"] = (admissions["ADMISSION_TYPE"] == "EMERGENCY").astype(int)
admissions["ADM_URGENT"] = (admissions["ADMISSION_TYPE"] == "URGENT").astype(int)

# --- Merge Demographics ---
demographics = admissions.merge(icustays, on=["SUBJECT_ID", "HADM_ID"], how="inner")

# # --- Process Prescriptions ---
# medications_list = ["amiodarone", "ampicillinsulbactam", "atropine", "calciumgluconate", "carvedilol", "cefazolin",
#                     "cefepime", "ceftriaxone", "clonazepam", "clopidogrel", "dextrose", "diazepam", "digoxin",
#                     "diltiazem", "diphenhydramine", "enoxaparin", "fentanyl", "fentanylcitrate", "fluconazole",
#                     "fondaparinux", "furosemide", "glucagon", "haloperidol", "heparin", "hydralazine",
#                     "hydromorphone", "insulin", "levofloxacin", "levothyroxine", "metoclopramide", "metoprolol",
#                     "metronidazole", "midazolam", "nitroglycerin", "nitroprusside", "norepinephrine", "ondansetron",
#                     "phenytoin", "piperacillin", "potassium_y", "prednisone", "propofol", "vancomycin"]
# prescriptions["drug"] = prescriptions["drug"].str.lower()  # Use lowercase "drug"
# filtered_prescriptions = prescriptions[prescriptions["drug"].isin(medications_list)] # Use lowercase "drug"


# # --- Add Medication Indicators ---
# for med in medications_list:
#     final_df[med] = final_df["HADM_ID"].isin(filtered_prescriptions["hadm_id"]).astype(int) # Use lowercase "hadm_id"

# --- Merge All Features ---
final_df = demographics.merge(pivot_vitals, on=["SUBJECT_ID", "HADM_ID", "ICUSTAY_ID"], how="left")

# --- Add GENDER ---
final_df = final_df.merge(patients, on="SUBJECT_ID", how="left")
final_df['GENDER_M'] = (final_df['GENDER'] == 'M').astype(int)
final_df['GENDER_F'] = (final_df['GENDER'] == 'F').astype(int)
final_df.drop(columns=['GENDER'], inplace=True)
# --- Add Age Category ---
final_df['ADMITTIME'] = pd.to_datetime(final_df['ADMITTIME'])
final_df['DOB'] = pd.to_datetime(final_df['DOB'], errors='coerce')

# Calculate age, handling potential NaT values and OverflowErrors
def calculate_age(row):
    admit_time = row['ADMITTIME']
    dob = row['DOB']
    if pd.notna(admit_time) and pd.notna(dob):
        try:
            age = (admit_time.to_pydatetime() - dob.to_pydatetime()).days / 365.25
            return age
        except OverflowError:
            return None  # Or a sentinel value like -1, depending on your needs
    else:
        return None

final_df['AGE'] = final_df.apply(calculate_age, axis=1)


def categorize_age(age):
    if pd.isna(age):
        return "Unknown"
    elif 40 <= age <= 64:
        return "AGE middle adult"
    elif age >= 65:
        return "AGE senior"
    else:
        return "Other"

final_df['Age Category'] = final_df['AGE'].apply(categorize_age)
final_df = pd.get_dummies(final_df, columns=['Age Category'], prefix='AGE', dummy_na=False)
final_df.drop(columns=['AGE', 'DOB'], inplace=True)



In [3]:
# --- Save ---
final_df.to_csv("../MIMIC_resources/ICU_patient_data.csv", index=False)
print("Data extraction complete. File saved as 'ICU_patient_data.csv'.")

final_df

Data extraction complete. File saved as 'ICU_patient_data.csv'.


,SUBJECT_ID,HADM_ID,ADMITTIME,ADMISSION_TYPE,ADM_ELECTIVE,ADM_EMERGENCY,ADM_URGENT,ICUSTAY_ID,Heart Rate,Systolic Blood Pressure,...,Anion Gap,Troponin,Platelets,INR,GENDER_M,GENDER_F,AGE_AGE middle adult,AGE_AGE senior,AGE_Other,AGE_Unknown
0,22,165315,2196-04-09 12:26:00,EMERGENCY,0,1,0,204798,NaN,NaN,...,NaN,NaN,NaN,NaN,0,1,False,False,True,False
1,23,152223,2153-09-03 07:15:00,ELECTIVE,1,0,0,227807,NaN,NaN,...,NaN,NaN,NaN,NaN,1,0,False,True,False,False
2,23,124321,2157-10-18 19:34:00,EMERGENCY,0,1,0,234044,77.0,67.0,...,15.0,NaN,NaN,1.0,1,0,False,True,False,False
3,24,161859,2139-06-06 16:14:00,EMERGENCY,0,1,0,262236,NaN,NaN,...,NaN,NaN,NaN,NaN,1,0,False,False,True,False
4,25,129635,2160-11-02 02:06:00,EMERGENCY,0,1,0,203487,NaN,NaN,...,NaN,NaN,NaN,NaN,1,0,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,98800,191113,2131-03-30 21:13:00,EMERGENCY,0,1,0,210188,122.0,120.0,...,6.0,NaN,NaN,1.3,0,1,False,False,True,False
61528,98802,101071,2151-03-05 20:00:00,EMERGENCY,0,1,0,294783,86.0,162.0,...,NaN,NaN,NaN,NaN,0,1,False,True,False,False
61529,98805,122631,2200-09-12 07:15:00,ELECTIVE,1,0,0,203155,112.0,160.0,...,15.0,NaN,NaN,1.1,1,0,True,False,False,False
61530,98813,170407,2128-11-11 02:29:00,EMERGENCY,0,1,0,283274,NaN,NaN,...,NaN,NaN,NaN,NaN,0,1,True,False,False,False


In [4]:
final_df.keys()

Index(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'ADMISSION_TYPE', 'ADM_ELECTIVE',
       'ADM_EMERGENCY', 'ADM_URGENT', 'ICUSTAY_ID', 'Heart Rate',
       'Systolic Blood Pressure', 'Diastolic Blood Pressure',
       'Respiratory Rate', 'Pulse Oximetry', 'PTT', 'WBC', 'RBC', 'Chloride',
       'Creatinine', 'Glucose', 'Sodium', 'Temperature (F)',
       'Inspired O2 Fraction', 'Tidal Volume', 'PEEP Set', 'BUN', 'Anion Gap',
       'Troponin', 'Platelets', 'INR', 'GENDER_M', 'GENDER_F',
       'AGE_AGE middle adult', 'AGE_AGE senior', 'AGE_Other', 'AGE_Unknown'],
      dtype='object')

In [6]:
final_df[['ADMISSION_TYPE', 'ADM_ELECTIVE',
       'ADM_EMERGENCY', 'ADM_URGENT', 'Heart Rate',
       'Systolic Blood Pressure', 'Diastolic Blood Pressure',
       'Respiratory Rate', 'Pulse Oximetry', 'PTT', 'WBC', 'RBC', 'Chloride',
       'Creatinine', 'Glucose', 'Sodium', 'Temperature (F)',
       'Inspired O2 Fraction', 'Tidal Volume', 'PEEP Set', 'BUN', 'Anion Gap',
       'Troponin', 'Platelets', 'INR', 'GENDER_M', 'GENDER_F',
       'AGE_AGE middle adult', 'AGE_AGE senior', 'AGE_Other', 'AGE_Unknown']]

,ADMISSION_TYPE,ADM_ELECTIVE,ADM_EMERGENCY,ADM_URGENT,Heart Rate,Systolic Blood Pressure,Diastolic Blood Pressure,Respiratory Rate,Pulse Oximetry,PTT,...,Anion Gap,Troponin,Platelets,INR,GENDER_M,GENDER_F,AGE_AGE middle adult,AGE_AGE senior,AGE_Other,AGE_Unknown
0,EMERGENCY,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,1,False,False,True,False
1,ELECTIVE,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,0,False,True,False,False
2,EMERGENCY,0,1,0,77.0,67.0,45.0,14.0,100.0,NaN,...,15.0,NaN,NaN,1.0,1,0,False,True,False,False
3,EMERGENCY,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,0,False,False,True,False
4,EMERGENCY,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,0,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,EMERGENCY,0,1,0,122.0,120.0,70.0,28.0,100.0,5.0,...,6.0,NaN,NaN,1.3,0,1,False,False,True,False
61528,EMERGENCY,0,1,0,86.0,162.0,92.0,29.0,100.0,5.0,...,NaN,NaN,NaN,NaN,0,1,False,True,False,False
61529,ELECTIVE,1,0,0,112.0,160.0,79.0,15.0,96.0,12.0,...,15.0,NaN,NaN,1.1,1,0,True,False,False,False
61530,EMERGENCY,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,1,True,False,False,False


In [ ]:
d_items['CATEGORY'].unique()

In [ ]:
DD = d_items[d_items['CATEGORY'] == 'Routine Vital Signs']
DD[['LABEL', 'ITEMID']]